In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

# Define your PyTorch models here
models = [MyModel1(), MyModel2(), MyModel3()]

# Define your loss function
criterion = nn.CrossEntropyLoss()

# Create a list to store the optimizer for each model
optimizers = [optim.Adam(model.parameters(), lr=0.001) for model in models]

# Create a dataframe to store the model accuracy, train loss, and test loss
df = pd.DataFrame(columns=['model', 'epoch', 'accuracy', 'train_loss', 'test_loss'])

# Train your models
num_epochs = 10

for model_idx, model in enumerate(models):
    for epoch in range(num_epochs):
        # Train the model
        model.train()
        train_loss = 0.0
        for inputs, labels in train_dataloader:
            optimizers[model_idx].zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizers[model_idx].step()
            train_loss += loss.item() * inputs.size(0)

        # Evaluate the model
        model.eval()
        total_correct = 0
        total_samples = 0
        test_loss = 0.0
        with torch.no_grad():
            for inputs, labels in test_dataloader:
                outputs = model(inputs)
                _, predictions = torch.max(outputs, 1)
                total_correct += (predictions == labels).sum().item()
                total_samples += labels.size(0)
                loss = criterion(outputs, labels)
                test_loss += loss.item() * inputs.size(0)
        accuracy = total_correct / total_samples
        train_loss = train_loss / len(train_dataloader.dataset)
        test_loss = test_loss / len(test_dataloader.dataset)
        print(f"Model {model_idx+1} - Epoch {epoch+1} Accuracy: {accuracy:.4f} Train Loss: {train_loss:.4f} Test Loss: {test_loss:.4f}")
        
        # Add the accuracy, train loss, and test loss to the dataframe
        df = df.append({'model': f'model{model_idx+1}', 'epoch': epoch+1, 'accuracy': accuracy, 'train_loss': train_loss, 'test_loss': test_loss}, ignore_index=True)

    # Save the dataframe as a CSV file for each model
    df.to_csv(f'model{model_idx+1}_accuracy.csv', index=False)
    df = df.iloc[0:0]

    # Generate the confusion matrix for the model on the test data
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predictions.cpu().numpy())
    cm = confusion_matrix(y_true, y_pred)
    report = classification_report(y_true, y_pred)
    print(f"Model {model_idx+1} Confusion Matrix:")
    print(cm)
    print(f"Classification Report:")
    print(report)